In [2]:
!pip install --user tensorflow

  Using cached https://files.pythonhosted.org/packages/5d/6a/9669836f813b73fe5abf5e9f118ccc9b7fb060f02789d385825b0943f9c8/tensorflow-2.3.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/a1/6b/7f62017e3f0b32438dd90bdc1ff0b7b1448b6cb04a1ed84f37b6de95cd7b/h5py-2.10.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/d6/84/759f5dd23fec8ba71952d97bcc7e2c9d7d63bdc582421f3cd4be845f0c98/gast-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/83/179c8f76e5716030cc3ee9433721161cfcc1d854e9ba20c9205180bb100a/tensorboard-2.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e4/01/7a26148f7de9eb6c27f95b29eba16b7e820827cb9ebaae182d7483e44711/numpy-1.18.5-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/e9/ed/5853ec0ae380cba4588eab1524e18ece1583b65f7ae0e97321f5ff9dfd60/tensorflow_estimator-2.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonh

  The script f2py.exe is installed in 'C:\Users\ramaz\AppData\Roaming\Python\Python37\Scripts' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The script markdown_py.exe is installed in 'C:\Users\ramaz\AppData\Roaming\Python\Python37\Scripts' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The scripts pyrsa-decrypt.exe, pyrsa-encrypt.exe, pyrsa-keygen.exe, pyrsa-priv2pub.exe, pyrsa-sign.exe and pyrsa-verify.exe are installed in 'C:\Users\ramaz\AppData\Roaming\Python\Python37\Scripts' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The script google-oauthlib-tool.exe is installed in 'C:\Users\ramaz\AppData\Roaming\Python\Python37\Scripts' which is not on PATH.
  Consider adding this directory to PATH or, if you pref

In [14]:
!pip install protobuf

In [29]:
!pip install protobuf-converter

  Running setup.py bdist_wheel for protobuf-converter: started
  Running setup.py bdist_wheel for protobuf-converter: finished with status 'done'
  Stored in directory: C:\Users\ramaz\AppData\Local\pip\Cache\wheels\84\91\cd\4186eff921eab47b616e662b2aa2c7cf970df95ed61f34c4ff
Successfully built protobuf-converter


In [35]:
! object_detection/protos/*.proto --python_out=. 

Usage: protoc [OPTION] PROTO_FILES
Parse PROTO_FILES and generate output based on the options given:
  -IPATH, --proto_path=PATH   Specify the directory in which to search for
                              imports.  May be specified multiple times;
                              directories will be searched in order.  If not
                              given, the current working directory is used.
                              If not found in any of the these directories,
                              the --descriptor_set_in descriptors will be
                              checked for required proto file.
  --version                   Show version info and exit.
  -h, --help                  Show this text and exit.
  --encode=MESSAGE_TYPE       Read a text-format message of the given type
                              from standard input and write it in binary
                              to standard output.  The message type must
                              be defined in PROTO_F

In [36]:
!pip install tensorflow-gpu

In [41]:
!pip install pillow

In [42]:
!pip install lxml

In [43]:
!pip install jupyter

In [44]:
!pip install matplotlib

In [ ]:
from PIL import Image
#from tflite_runtime.interpreter import Interpreter
from urllib import request
import datetime
import io
import csv
import numpy as np
import os
import paho.mqtt.client as mqtt_client
import picamera
import re
import sys
import time

CAMERA_WIDTH = 500
CAMERA_HEIGHT = 500
THRESHOLD = 0.6

MODEL_PATH = os.sep.join([os.getcwd(), "detect.tflite"])
LABELS_PATH = os.sep.join([os.getcwd(), "coco_labels.txt"])
FILE_NAME = os.sep.join([os.getcwd(), "history.csv"])

def load_labels(path):
  """Loads the labels file. Supports files with or without index numbers."""
  with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    labels = {}
    for row_number, content in enumerate(lines):
      pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
      if len(pair) == 2 and pair[0].strip().isdigit():
        labels[int(pair[0])] = pair[1].strip()
      else:
        labels[row_number] = pair[0].strip()
  return labels


def set_input_tensor(interpreter, image):
  """Sets the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
  """Returns the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  set_input_tensor(interpreter, image)
  interpreter.invoke()

  # Get all output details
  boxes = get_output_tensor(interpreter, 0)
  timestamp = str(datetime.datetime.now()).split(".")[0]
  classes = get_output_tensor(interpreter, 1)
  scores = get_output_tensor(interpreter, 2)
  count = int(get_output_tensor(interpreter, 3))

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
          'bounding_box': boxes[i],
          'class_id': classes[i],
          'score': scores[i],
          'timestamp': timestamp,
      }
      results.append(result)
  return results

# on connetc indication of the client
def on_connect(client, userdata, flags, rc):
    if(rc == 0):
        print("Connection success")
    else:
        print("Connection failed")

# function to check if pi is connected to internert
def connected_to_internet(host="http://google.com"):
    try:
        request.urlopen(host)
        return True
    except:
        return False

def write(info):
    with open(FILE_NAME, "a+") as fout:
        write = csv.writer(fout)
        write.writerows(info)

def read():
    data = []
    with open(FILE_NAME, "r") as fout:
        reader = csv.reader(fout)
        for row in reader:
            data.append(row)
    return data

def main():
    
    # set up mqtt publisher role
    publisher = mqtt_client.Client()
    publisher.on_connect = on_connect
        
    # load coco labels
    labels = load_labels(LABELS_PATH)
    
    # load interpreter
    interpreter = Interpreter(MODEL_PATH)
    interpreter.allocate_tensors()
    
    shape_dimensions = interpreter.get_input_details()[0]['shape']
    
    height, width = shape_dimensions[1], shape_dimensions[2]
    
    # keep track if historical data exists
    HISTORICAL_DATA = False
    
    # set up PI camera
    with picamera.PiCamera(
        resolution=(CAMERA_WIDTH, CAMERA_HEIGHT), framerate=30
    ) as camera:
        camera.rotation = 180
        
        # start camera
        camera.start_preview()
        # store each continous capture in memory for faster computation
        stream = io.BytesIO()
        
        counter = 0
        
        for capture in camera.capture_continuous(stream, format="jpeg", use_video_port=True):
            stream.seek(0)
            image = Image.open(stream).convert("RGB").resize(
                (height, width), Image.ANTIALIAS)
            # call detect objects
            results = detect_objects(interpreter, image, THRESHOLD)
            out = []
            # make sure results are not empty
            if len(results) > 0:
                for result in results:
                    temp = [
                        'timestamp' + ": " + result['timestamp'],
                        'class_id' + ": " + labels[result['class_id']],
                    ]
                    out.append(temp)
                if connected_to_internet():
                    publisher.connect("broker.emqx.io", 1883, 60)
                    if HISTORICAL_DATA:
                        # maintain the time stamp order if historical data exists
                        data = read()
                        for i in out:
                            data.append(i)
                        out = data
                        
                        # clear file contents since everything will be published
                        f = open(FILE_NAME, "w+")
                        f.close()
                    
                    HISTORICAL_DATA = False
                    for i in out:
                        publisher.publish('raspberry/meta', payload=str(i), qos=0, retain=False)
                        print("sent ", i, " to raspberry/temp")
                else:
                    print("Not connected to internet")
                    HISTORICAL_DATA = True
                    write(out)
            out.clear()
            stream.seek(0)
            stream.truncate()
            time.sleep(3)